# Make Demographic, ID Characteristic, & Feature Category Tables

Gaylen Fronk  
May 23, 2024

In [ ]:

# packages for script
library(tidyverse)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

ℹ SHA-1 hash of file is "c045eee2655a18dc85e715b78182f176327358a7"

In [ ]:

# handle conflicts
options(conflicts.policy = "depends.ok")


In [ ]:
# absolute paths
switch (Sys.info()[['sysname']],
        # PC paths
        Windows = {
          path_data <- str_c("P:/studydata/match/processed_data/")},
        
        # IOS paths
        Darwin = {
          path_data <- str_c("/Volumes/private/studydata/match/processed_data/")},
        
        # Linux paths
        Linux = {
          path_data <- str_c("~/mnt/private/studydata/match/processed_data/")}
)


In [ ]:
# chunk defaults

options(tibble.width = Inf)
options(tibble.print_max = Inf)


## Read in processed data

In [ ]:
d <- read_csv(file.path(path_data, "match_cln.csv"),
              show_col_types = FALSE)


## Table 1: Demographic Characteristics

Select down data

In [ ]:
dt1 <- d |> 
  select(employment, ends_with("_ehr"), income_order) |> 
  select(-bmi_ehr, -diabetic_ehr) |> 
  glimpse()


Rows: 1,086
Columns: 7
$ employment         <chr> "unemployed", "employed_full_time", "employed_full_…
$ gender_ehr         <chr> "female", "female", "female", "female", "female", "…
$ age_ehr            <dbl> 49, 38, 51, 55, 65, 50, 48, 47, 45, 47, 52, 70, 48,…
$ race_ehr           <chr> "white", "white", "white", "white", "white", "white…
$ marital_status_ehr <chr> "divorced", "married", "divorced", "married", "marr…
$ ethnicity_ehr      <chr> "non_hispanic", "non_hispanic", "non_hispanic", "no…
$ income_order       <chr> "01_less_than_10000", "07_more_than_75000", "06_btw…

Build table

In [ ]:
tab_race <- dt1 |> 
  tab(race_ehr) |> 
  mutate(race_ehr = case_match(
    race_ehr,
    "asian" ~ "   Asian",
    "black_african_american" ~ "   Black/African American",
    "multiracial" ~ "   Multiracial",
    "native_american_alaska_native" ~ "   Native American/Alaska Native",
    "other" ~ "   Other",
    "white" ~ "   White"
  )) |> 
  rename(var_name = race_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Race",
          n_prop = "",
          .before = 1)

tab_ethnicity <- dt1 |> 
  tab(ethnicity_ehr) |> 
  mutate(ethnicity_ehr = case_match(
    ethnicity_ehr,
    "hispanic_latinx" ~ "   Hispanic or Latino/a",
    "non_hispanic" ~ "   Non-Hispanic"
  )) |> 
  rename(var_name = ethnicity_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Ethnicity",
          n_prop = "",
          .before = 1)

tab_gender <- dt1 |> 
  tab(gender_ehr) |> 
  mutate(gender_ehr = case_match(
    gender_ehr,
    "female" ~ "   Female",
    "male" ~ "   Male"
  )) |> 
  rename(var_name = gender_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Gender",
          n_prop = "",
          .before = 1)

tab_employment <- dt1 |> 
  tab(employment) |> 
  mutate(employment = case_match(
    employment,
    "employed_full_time" ~ "   Employed (full-time)",
    "employed_part_time" ~ "   Employed (part-time)",
    "unemployed" ~ "   Unemployed"
  )) |> 
  rename(var_name = employment) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Employment",
          n_prop = "",
          .before = 1)

tab_marital <- dt1 |> 
  tab(marital_status_ehr) |> 
  mutate(marital_status_ehr = case_match(
    marital_status_ehr,
    "divorced" ~ "   Divorced",
    "live_with_domestic_partner" ~ "   Living with a domestic partner",
    "married" ~ "   Married",
    "separated" ~ "   Separated",
    "widowed" ~ "   Widowed",
    "never_married" ~ "   Never married",
    NA ~ "   Did not respond"
  )) |> 
  rename(var_name = marital_status_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Marital Status",
          n_prop = "",
          .before = 1)

tab_income <- dt1 |> 
  tab(income_order) |> 
  mutate(income_order = case_match(
    income_order,
    "01_less_than_10000" ~ "   < $10,000",
    "02_btwn_10000_19999" ~ "   $10,000 - $19,999",
    "03_btwn_20000_24999" ~ "   $20,000 - $24,999",
    "04_btwn_25000_34999" ~ "   $25,000 - $34,999",
    "05_btwn_35000_49999" ~ "   $35,000 - $49,999",
    "06_btwn_50000_74999" ~ "   $50,000 - $74,999",
    "07_more_than_75000" ~ "   $75,000+",
    NA ~ "   Did not respond"
  )) |> 
  rename(var_name = income_order) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Income",
          n_prop = "",
          .before = 1)

tab_age <- tibble(
  var_name = "Age",
  n_prop = "",
  `Mean (SD)` = str_c(round(mean(dt1$age_ehr), digits = 2),
                      " (", round(sd(dt1$age_ehr), digits = 2), ")")
)

t1_demo <- bind_rows(tab_gender, tab_race) |> 
  bind_rows(tab_ethnicity) |> 
  bind_rows(tab_marital) |> 
  bind_rows(tab_employment) |> 
  bind_rows(tab_income) |> 
  mutate(`Mean (SD)` = "") |> 
  add_row(tab_age, .before = 1) |> 
  rename(Characteristic = var_name,
         `N (%)` = n_prop)


Table 1

``` r
print(t1_demo)
```

    # A tibble: 35 × 3
       Characteristic                      `N (%)`         `Mean (SD)`   
       <chr>                               <chr>           <chr>         
     1 "Age"                               ""              "48.13 (11.6)"
     2 "Gender"                            ""              ""            
     3 "   Female"                         "566 (52.12%)"  ""            
     4 "   Male"                           "520 (47.88%)"  ""            
     5 "Race"                              ""              ""            
     6 "   Asian"                          "3 (0.28%)"     ""            
     7 "   Black/African American"         "309 (28.45%)"  ""            
     8 "   Multiracial"                    "22 (2.03%)"    ""            
     9 "   Native American/Alaska Native"  "6 (0.55%)"     ""            
    10 "   Other"                          "18 (1.66%)"    ""            
    11 "   White"                          "728 (67.03%)"  ""            
    12 "Ethnicity"                         ""              ""            
    13 "   Hispanic or Latino/a"           "27 (2.49%)"    ""            
    14 "   Non-Hispanic"                   "1059 (97.51%)" ""            
    15 "Marital Status"                    ""              ""            
    16 "   Divorced"                       "224 (20.63%)"  ""            
    17 "   Living with a domestic partner" "87 (8.01%)"    ""            
    18 "   Married"                        "384 (35.36%)"  ""            
    19 "   Never married"                  "299 (27.53%)"  ""            
    20 "   Separated"                      "53 (4.88%)"    ""            
    21 "   Widowed"                        "34 (3.13%)"    ""            
    22 "   Did not respond"                "5 (0.46%)"     ""            
    23 "Employment"                        ""              ""            
    24 "   Employed (full-time)"           "474 (43.65%)"  ""            
    25 "   Employed (part-time)"           "283 (26.06%)"  ""            
    26 "   Unemployed"                     "329 (30.29%)"  ""            
    27 "Income"                            ""              ""            
    28 "   < $10,000"                      "210 (19.34%)"  ""            
    29 "   $10,000 - $19,999"              "135 (12.43%)"  ""            
    30 "   $20,000 - $24,999"              "83 (7.64%)"    ""            
    31 "   $25,000 - $34,999"              "129 (11.88%)"  ""            
    32 "   $35,000 - $49,999"              "149 (13.72%)"  ""            
    33 "   $50,000 - $74,999"              "160 (14.73%)"  ""            
    34 "   $75,000+"                       "167 (15.38%)"  ""            
    35 "   Did not respond"                "53 (4.88%)"    ""            

## Table 2: Smoking-related ID Characteristics

Select down data

In [ ]:
dt2 <- d |> 
  select(age_1st_cig, age_daily_smoker, years_smoke, cigs_per_day_heaviest,
         tot_quit_attempt, cigs_per_day, dsm5_total_scale, ftnd_1_item_order,
         wisdm37_total_scale, wsws_total_scale)


Build table

In [ ]:
tab_ftnd_1 <- dt2 |> 
  tab(ftnd_1_item_order) |> 
  mutate(ftnd_1_item_order = case_match(
    ftnd_1_item_order,
    "03_within_5_min" ~ "   Within 5 minutes",
    "02_6_to_30_min" ~ "   6 - 30 minutes",
    "01_31_to_60_min" ~ "   31 - 60 minutes",
    "00_after_60_min" ~ "   After 60 minutes",
    NA ~ "   Did not respond"
  )) |> 
  rename(var_name = ftnd_1_item_order) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Time to first cigarette upon waking",
          n_prop = "",
          .before = 1) |> 
  rename(`N (%)` = n_prop, Characteristic = var_name) |> 
  mutate(`Mean (SD)` = "")

tab2_means <- dt2 |> 
  select(-ftnd_1_item_order) |> 
  summarize(across(everything(), ~ mean(.x, na.rm = TRUE))) |> 
  pivot_longer(
    cols = everything(),
    names_to = "var_name",
    values_to = "means"
  ) |> 
  mutate(means = round(means, digits = 2))

tab2_sds <- dt2 |> 
  select(-ftnd_1_item_order) |> 
  summarize(across(everything(), ~ sd(.x, na.rm = TRUE))) |> 
  pivot_longer(
    cols = everything(),
    names_to = "var_name",
    values_to = "sds"
  ) |> 
  mutate(sds = round(sds, digits = 2))

t2_idchars <- full_join(tab2_means, tab2_sds, by = "var_name") |> 
  mutate(var_name = case_match(
    var_name,
    "age_1st_cig" ~ "Age of first cigarette",
    "age_daily_smoker" ~ "Age became daily smoker",
    "years_smoke" ~ "Years smoking",
    "cigs_per_day" ~ "Cigarettes per day (current)",
    "cigs_per_day_heaviest" ~ "Cigarettes per day (heaviest)",
    "tot_quit_attempt" ~ "Number of previous quit attempts",
    "dsm5_total_scale" ~ "Number of DSM5 tobacco use disorder symptoms",
    "wisdm37_total_scale" ~ "WISDM37 Total Score",
    "wsws_total_scale" ~ "WSWS Total Score"
  )) |> 
  mutate(`Mean (SD)` = str_c(means, " (", sds, ")"),
         `N (%)` = "") |> 
  rename(Characteristic = var_name) |> 
  select(-means, -sds) |> 
  full_join(tab_ftnd_1,
            by = join_by(Characteristic, `Mean (SD)`, `N (%)`))


Table 2

``` r

print(t2_idchars)
```

    # A tibble: 15 × 3
       Characteristic                                 `Mean (SD)`     `N (%)`       
       <chr>                                          <chr>           <chr>         
     1 "Age of first cigarette"                       "14.6 (3.42)"   ""            
     2 "Age became daily smoker"                      "17.56 (4.62)"  ""            
     3 "Years smoking"                                "28.65 (12.03)" ""            
     4 "Cigarettes per day (heaviest)"                "22.74 (10.67)" ""            
     5 "Number of previous quit attempts"             "3.91 (6.03)"   ""            
     6 "Cigarettes per day (current)"                 "17.03 (8.31)"  ""            
     7 "Number of DSM5 tobacco use disorder symptoms" "4.57 (2.17)"   ""            
     8 "WISDM37 Total Score"                          "3.99 (1.17)"   ""            
     9 "WSWS Total Score"                             "48.82 (19.91)" ""            
    10 "Time to first cigarette upon waking"          ""              ""            
    11 "   After 60 minutes"                          ""              "82 (7.55%)"  
    12 "   31 - 60 minutes"                           ""              "157 (14.46%)"
    13 "   6 - 30 minutes"                            ""              "477 (43.92%)"
    14 "   Within 5 minutes"                          ""              "366 (33.7%)" 
    15 "   Did not respond"                           ""              "4 (0.37%)"   

## Table 3: Feature Categories

Set up data

In [ ]:
dt3 <- d |> select(-contains("outcome"), -subid)


Build table

In [ ]:
t3_feat_misc <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Treatment", "Incarcerated", "Works Third Shift"),
  type = c("Categorical", "Dichotomous", "Dichotomous"),
  n_feat = c(1, 1, 1)
) |> 
  add_row(feat_cat = "Miscellaneous",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t3_feat_psych_sud <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Frequency of Drinking Alcohol", "Quantity of Alcohol", "Binge Drinking", "Short Inventory of Problems-2 (Revised)", "Life Satisfaction", "Life Enjoyment", "Psychological Disorder Diagnoses", "Positive and Negative Affect Schedule", "Snaith-Hamilton Pleasure Scale", "Anxiety Severity Index-3", "Distress Tolerance Scale", "Patient History Questionnaire-9"),
  type = c("Ordinal", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Dichotomous", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Ordinal"),
  n_feat = c(1, 1, 1, 16, 1, 1, 7, 8, 15, 22, 20, 11)
) |> 
  add_row(feat_cat = "Mental Health & Psychological Traits",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t3_feat_smoking <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Baseline Carbon Monoxide", "Carbon Monoxide Exposure", "Age of 1st Cigarette", "Age Became Daily Smoker", "Years Smoking", "Cigarettes Per Day (Heaviest)", "Use of Other Tobacco Products", "Number of Previous Quit Attempts", "Last Recent Quit Attempt", "Longest Quit Attempt", "Previous Quit Methods Used", "Cigarettes Per Day (Current)", "Motivation to Quit", "Self-Efficacy for Quitting in Next 30 Days", "Confidence to Quit", "Importance to Quit", "DSM5 Tobacco Use Disorder", "Fagerstrom Test of Nicotine Dependence", "Wisconsin Inventory of Smoking Dependence Motives-37", "Smoke Menthol Cigarettes", "Wisconsin Smoking Withdrawal Scale-2"),
  type = c("Numeric", "Dichotomous", "Numeric", "Numeric", "Numeric", "Numeric", "Categorical (Unordered)", "Numeric", "Ordinal", "Ordinal", "Dichotomous", "Numeric", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Dichotomous", "Dichotomous, Ordinal", "Ordinal", "Categorical (Unordered)", "Ordinal"),
  n_feat = c(1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 6, 1, 1, 1, 1, 1, 14, 6, 49, 1, 45)
) |> 
  add_row(feat_cat = "Smoking Use & History",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t3_feat_medical <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Diabetes Diagnosis", "Multidimensional Fatigue Inventory", "Berlin Sleep Questionnaire", "Body Mass Index", "Health-related Quality of Life Scale", "Healthy Days Symptoms Module"),
  type = c("Dichotomous", "Ordinal", "Ordinal, Dichotomous", "Numeric", "Ordinal, Numeric", "Numeric"),
  n_feat = c(1, 26, 3, 1, 4, 5)
) |> 
  add_row(feat_cat = "Medical & Physical Health",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t3_feat_social_environ <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Spouse Smokes", "Live with Another Smoker", "People Close to You Who Smoke", "Ban on Smoking at Home", "Ban on Smoking at Work", "Time Around Other Smokers"),
  type = c("Categorical (unordered)", "Categorical (unordered)", "Categorical (unordered)", "Dichotomous", "Categorical (unordered)", "Ordinal"),
  n_feat = c(1, 1, 5, 1, 1, 2)
) |> 
  add_row(feat_cat = "Social & Environmental Characteristics",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t3_feat_demo <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Gender", "Age", "Race", "Marital Status", "Income", "Ethnicity", "Employment"),
  type = c("Categorical (unordered)", "Numeric", "Categorical (unordered)", "Categorical (unordered)", "Ordinal", "Categorical (Unordered)", "Categorical (unordered)"),
  n_feat = c(1, 1, 1, 1, 1, 1, 1)
) |> 
  add_row(feat_cat = "Demographic Characteristics",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t3_feat <- bind_rows(t3_feat_demo, t3_feat_smoking) |> 
  bind_rows(t3_feat_social_environ) |> 
  bind_rows(t3_feat_psych_sud) |> 
  bind_rows(t3_feat_medical) |> 
  bind_rows(t3_feat_misc) |> 
  rename(Category = feat_cat,
         `Feature Name` = feat_name,
         Type = type,
         `Number of Features` = n_feat)


Table 3

``` r
print(t3_feat)
```

    # A tibble: 61 × 4
       Category                              
       <chr>                                 
     1 Demographic Characteristics           
     2 <NA>                                  
     3 <NA>                                  
     4 <NA>                                  
     5 <NA>                                  
     6 <NA>                                  
     7 <NA>                                  
     8 <NA>                                  
     9 Smoking Use & History                 
    10 <NA>                                  
    11 <NA>                                  
    12 <NA>                                  
    13 <NA>                                  
    14 <NA>                                  
    15 <NA>                                  
    16 <NA>                                  
    17 <NA>                                  
    18 <NA>                                  
    19 <NA>                                  
    20 <NA>                                  
    21 <NA>                                  
    22 <NA>                                  
    23 <NA>                                  
    24 <NA>                                  
    25 <NA>                                  
    26 <NA>                                  
    27 <NA>                                  
    28 <NA>                                  
    29 <NA>                                  
    30 <NA>                                  
    31 Social & Environmental Characteristics
    32 <NA>                                  
    33 <NA>                                  
    34 <NA>                                  
    35 <NA>                                  
    36 <NA>                                  
    37 <NA>                                  
    38 Mental Health & Psychological Traits  
    39 <NA>                                  
    40 <NA>                                  
    41 <NA>                                  
    42 <NA>                                  
    43 <NA>                                  
    44 <NA>                                  
    45 <NA>                                  
    46 <NA>                                  
    47 <NA>                                  
    48 <NA>                                  
    49 <NA>                                  
    50 <NA>                                  
    51 Medical & Physical Health             
    52 <NA>                                  
    53 <NA>                                  
    54 <NA>                                  
    55 <NA>                                  
    56 <NA>                                  
    57 <NA>                                  
    58 Miscellaneous                         
    59 <NA>                                  
    60 <NA>                                  
    61 <NA>                                  
       `Feature Name`                                       Type                   
       <chr>                                                <chr>                  
     1 <NA>                                                 <NA>                   
     2 Gender                                               Categorical (unordered)
     3 Age                                                  Numeric                
     4 Race                                                 Categorical (unordered)
     5 Marital Status                                       Categorical (unordered)
     6 Income                                               Ordinal                
     7 Ethnicity                                            Categorical (Unordered)
     8 Employment                                           Categorical (unordered)
     9 <NA>                                                 <NA>                   
    10 Baseline Carbon Monoxide                             Numeric                
    11 Carbon Monoxide Exposure                             Dichotomous            
    12 Age of 1st Cigarette                                 Numeric                
    13 Age Became Daily Smoker                              Numeric                
    14 Years Smoking                                        Numeric                
    15 Cigarettes Per Day (Heaviest)                        Numeric                
    16 Use of Other Tobacco Products                        Categorical (Unordered)
    17 Number of Previous Quit Attempts                     Numeric                
    18 Last Recent Quit Attempt                             Ordinal                
    19 Longest Quit Attempt                                 Ordinal                
    20 Previous Quit Methods Used                           Dichotomous            
    21 Cigarettes Per Day (Current)                         Numeric                
    22 Motivation to Quit                                   Ordinal                
    23 Self-Efficacy for Quitting in Next 30 Days           Ordinal                
    24 Confidence to Quit                                   Ordinal                
    25 Importance to Quit                                   Ordinal                
    26 DSM5 Tobacco Use Disorder                            Dichotomous            
    27 Fagerstrom Test of Nicotine Dependence               Dichotomous, Ordinal   
    28 Wisconsin Inventory of Smoking Dependence Motives-37 Ordinal                
    29 Smoke Menthol Cigarettes                             Categorical (Unordered)
    30 Wisconsin Smoking Withdrawal Scale-2                 Ordinal                
    31 <NA>                                                 <NA>                   
    32 Spouse Smokes                                        Categorical (unordered)
    33 Live with Another Smoker                             Categorical (unordered)
    34 People Close to You Who Smoke                        Categorical (unordered)
    35 Ban on Smoking at Home                               Dichotomous            
    36 Ban on Smoking at Work                               Categorical (unordered)
    37 Time Around Other Smokers                            Ordinal                
    38 <NA>                                                 <NA>                   
    39 Frequency of Drinking Alcohol                        Ordinal                
    40 Quantity of Alcohol                                  Ordinal                
    41 Binge Drinking                                       Ordinal                
    42 Short Inventory of Problems-2 (Revised)              Ordinal                
    43 Life Satisfaction                                    Ordinal                
    44 Life Enjoyment                                       Ordinal                
    45 Psychological Disorder Diagnoses                     Dichotomous            
    46 Positive and Negative Affect Schedule                Ordinal                
    47 Snaith-Hamilton Pleasure Scale                       Ordinal                
    48 Anxiety Severity Index-3                             Ordinal                
    49 Distress Tolerance Scale                             Ordinal                
    50 Patient History Questionnaire-9                      Ordinal                
    51 <NA>                                                 <NA>                   
    52 Diabetes Diagnosis                                   Dichotomous            
    53 Multidimensional Fatigue Inventory                   Ordinal                
    54 Berlin Sleep Questionnaire                           Ordinal, Dichotomous   
    55 Body Mass Index                                      Numeric                
    56 Health-related Quality of Life Scale                 Ordinal, Numeric       
    57 Healthy Days Symptoms Module                         Numeric                
    58 <NA>                                                 <NA>                   
    59 Treatment                                            Categorical            
    60 Incarcerated                                         Dichotomous            
    61 Works Third Shift                                    Dichotomous            
       `Number of Features`
                      <dbl>
     1                   NA
     2                    1
     3                    1
     4                    1
     5                    1
     6                    1
     7                    1
     8                    1
     9                   NA
    10                    1
    11                    1
    12                    1
    13                    1
    14                    1
    15                    1
    16                    5
    17                    1
    18                    1
    19                    1
    20                    6
    21                    1
    22                    1
    23                    1
    24                    1
    25                    1
    26                   14
    27                    6
    28                   49
    29                    1
    30                   45
    31                   NA
    32                    1
    33                    1
    34                    5
    35                    1
    36                    1
    37                    2
    38                   NA
    39                    1
    40                    1
    41                    1
    42                   16
    43                    1
    44                    1
    45                    7
    46                    8
    47                   15
    48                   22
    49                   20
    50                   11
    51                   NA
    52                    1
    53                   26
    54                    3
    55                    1
    56                    4
    57                    5
    58                   NA
    59                    1
    60                    1
    61                    1